# Getting Started With Amazon Lookout for Metrics

Amazon Lookout for Metrics can help you identify anomalies within your data regardless of its origin. By following this notebook you will build out a solution using Amazon Lookout for Metrics to capture incoming data and to detect anomalies within it.

This guide assumes you completed all the work in `README.md` and the prerequisites in the `1.PrereqSetupData.ipynb`. If you have not, go back and complete it first. If you have not, go back to that first then return. Like any good notebook, this one will wait patiently for you to do so.

## Amazon Lookout for Metrics's Workflow

1. Create a Detector and configure its detection properties.
2. Create a Metric Set:
    1. Provide the location of your source data and the IAM permissions needed to access it. 
    1. Define the Metrics that you want to investigate.
    1. Attach the dataset to your Detector.
3. Activate the Detector.
4. (Optional) Set up alerts to get notified when L4M detects important outliers.
5. Inspect the detected outliers to figure out their root causes.
6. Provide feedback on the outliers to improve predictor accuracy.

First restore the variables from the previous notebook and then import the libraries needed.

In [ ]:
%store -r

Next, import the new boto3 and a few other packages, then establish a connection to AWS:

In [ ]:
import json
import time

import boto3
import utility

**IF THE CELL BELOW GENERATES AN ERROR:** This is totally normal, it may mean that your version of Boto3 is simply out of date. To correct this go to the cell below the one that errored and run it to update to the latest version of Boto3 inside SageMaker. IF you are not using a SageMaker Notebook simply follow the instructions for your Python environment. 

**DO NOT RUN THE UPGRADE CELL UNLESS YOU NEED TO**

In [ ]:
L4M = boto3.client( "lookoutmetrics", region_name="us-west-2" )

In [ ]:
# THIS IS OPTIONAL, DO NOT RUN IT IF YOU DO NOT NEED TO 
!pip install --upgrade boto3

After that, validate that you are receiving anything back from this API call below, if you get a 200 code it means you are allow-listed and connecting to the service successfully!

In [ ]:
L4M.list_metric_sets()

## Creating A Detector

Now the basic external resources are ready, so it is time to get started with Amazon Lookout for Metrics, that starts with creating a `Detector`.

### Detectors

To detect outliers, Amazon Lookout for Metrics builds a machine learning model that is trained with your source data. This model, called a `Detector`, is automatically trained with the machine learning algorithm that best fits your data and use case. You can either provide your historical data for training, if you have any, or get started with real-time data, and Amazon Lookout for Metrics will learn on-the-go. 

You specify the Amazon S3 location that Amazon Lookout for Metrics should continuously monitor for new data, and your detector analyzes your data and returns information about the outliers that it detected. When you create a `Detector`, you also specify a `detecting domain` and an `outlier detection frequency`. 

The `anomaly detection frequency` specifies how frequently the detector should wake-up and look for new data, run analysis and alert you with any interesting findings.

In [ ]:
project = "initial-lookoutmetrics-live-test" # just a string used to name resources such as MetricSet, Detector, etc.

frequency = "PT1H" # one of 'P1D', 'PT1H', 'PT10M' and 'PT5M'

In [ ]:
response = L4M.create_anomaly_detector( 
    AnomalyDetectorName = project + "-detector",
    AnomalyDetectorDescription = "My Detector",
    AnomalyDetectorConfig = {
        "AnomalyDetectorFrequency" : frequency,
    },
)

anomaly_detector_arn = response["AnomalyDetectorArn"]
anomaly_detector_arn

See details of created detector and it's status (should be `INACTIVE` at this point)

In [ ]:
L4M.describe_anomaly_detector(AnomalyDetectorArn=anomaly_detector_arn)

## Define Metrics

### Measures and Dimensions

`Measures` are variables or key performance indicators on which customers want to detect outliers and `Dimensions` are meta-data that represent categorical information about the measures. 

In this E-commerce example, views and revenue are our measures and platform and marketplace are our dimensions. Customers may want to monitor their data for anomalies in number of views or revenue for every platform, marketplace, and combination of both. You can designate up to five measures and five dimensions per dataset.

### Metrics 


After creating a detector, and mapping your measures and dimensions, Amazon Lookout for Metrics will analyze each combination of these measures and dimensions. For the above example, we have of 7 unique values (us, jp, de, etc.) for marketplace and 3 unique values (mobile web, mobile app, pc web) for platform for a total of 21 unique combinations. Each unique combination of measures with the dimension values (e.g. us/mobile app/revenue) is a time series `metric`. In this case, we have 21 dimensions and 2 measures for a total of 42 time-series `metrics`. 

Amazon Lookout for Metrics detects anomalies at the most granular level so you are able to pin-point any unexpected behavior in your data.

### Datasets

Measures, dimensions and metrics map to `datasets`, which also contain the Amazon S3 locations of your source data, an IAM role that has both read and write permissions to those Amazon S3 locations, and the rate at which data should be ingested from the source location (the upload frequency and data ingestion delay).


Now, create a metric set for your detector that points to the Live data in S3:

In [ ]:
s3_path_format = 's3://'+ s3_bucket + '/ecommerce/live/{{yyyyMMdd}}/{{HHmm}}'
s3_path_format

To kick this off faster, uncomment out the 3 lines for `HistoricalDataPathList` and fill in the value from notebook 3, for example  with `"s3://059124553121-lookoutmetrics-lab/ecommerce/backtest/"` as the only value in the list.

In [ ]:
params = {
    "AnomalyDetectorArn": anomaly_detector_arn,
    "MetricSetName" : project + '-metric-set-1',
    "MetricList" : [
        {
            "MetricName" : "views",
            "AggregationFunction" : "SUM",
        },
        {
            "MetricName" : "revenue",
            "AggregationFunction" : "SUM",
        },
    ],

    "DimensionList" : [ "platform", "marketplace" ],

    "TimestampColumn" : {
        "ColumnName" : "timestamp",
        "ColumnFormat" : "yyyy-MM-dd HH:mm:ss",
    },

    #"Delay" : 120, # seconds the detector will wait before attempting to read latest data per current time and detection frequency below
    "MetricSetFrequency" : frequency,

    "MetricSource" : {
        "S3SourceConfig": {
            "RoleArn" : role_arn,
#            "HistoricalDataPathList": [
#                s3_path_training_prefix,
#            ],
            "TemplatedPathList": [
                s3_path_format,
            ],

            "FileFormatDescriptor" : {
                "CsvFormatDescriptor" : {
                    "FileCompression" : "NONE",
                    "Charset" : "UTF-8",
                    "ContainsHeader" : True,
                    "Delimiter" : ",",
#                    "HeaderList" : [
#                        "platform",
#                        "marketplace",
#                        "timestamp",
#                        "views",
#                        "revenue"
#                    ],
                    "QuoteSymbol" : '"'
                },
            }
        }
    },
}

params

In [ ]:
response = L4M.create_metric_set( ** params )

metric_set_arn = response["MetricSetArn"]
metric_set_arn

To see that the metric set was created correctly:

In [ ]:
L4M.describe_metric_set(MetricSetArn=metric_set_arn)

## Activate the Detector

Now that the MetricSet has been specified, we are ready to start training the detector, that's done by activating it. The activation process can take a few minutes, feel free to strech your arms, go for a quick coffee.

In [ ]:
L4M.activate_anomaly_detector(AnomalyDetectorArn = anomaly_detector_arn)

In [ ]:
utility.wait_anomaly_detector( L4M, anomaly_detector_arn )

## Create Anomaly Alerts

Once your detector is active, you can attach alerts to it. `Alerts` are customized notifications available via the Amazon Simple Notification Service (SNS), configurable directly via the Amazon Lookout for Metrics console and SDK. These alerts notify you whenever an anomaly of a specified severity level is detected. Severity levels are a measure of the urgency or criticality of detected outliers. Alerts are meant to guide you towards relative prioritization of the outliers. Amazon Lookout for Metrics supports Critical, High, Medium, and Low thresholds. For example, you can set an alert on your detector to notify you whenever an outlier with a Medium severity level or greater is detected.


Before we get to creating an alert, 2 additional things are needed:

1. A role giving lookoutmetrics access to SNS
2. An SNS topic to deliver the alerts to

The cells below will guide you through creating those and then it is time to create the alert.


In [ ]:
iam = boto3.client("iam")
role_name = "L4M-SNSFullAccessCF"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "lookoutmetrics.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

try:
    create_role_response = iam.create_role(
        RoleName = role_name,
        AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
    )
    sns_role_arn = create_role_response["Role"]["Arn"]
    print( "Created", role_arn )
except iam.exceptions.EntityAlreadyExistsException:
    print("Role %s already existed" % role_name )
    sns_role_arn = boto3.resource('iam').Role(role_name).arn
    
print("Attaching policies")

# Now add SNS support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonSNSFullAccess',
    RoleName=role_name
)

print("Waiting for a minute to allow IAM role policy attachment to propagate")
time.sleep(60)

sns_role_arn

Now create the SNS topic for the alerts:

**UPDATE YOUR CELL NUMBER BELOW!!!**

In [ ]:
sns_client = boto3.client("sns")
topic = sns_client.create_topic(Name="anomalyalertsCF")
topic_arn = topic['TopicArn']

# Change to your cell
number = "+1xxx555xxxx" # Change to your cell

sns_client.subscribe(
        TopicArn=topic_arn,
        Protocol='sms',
        Endpoint=number  
)

The last step is to execute the cell below to configure the alerts to notify your topic.

In [ ]:
response = L4M.create_alert(
    Action = {
      "SNSConfiguration": { 
         "RoleArn": sns_role_arn,
         "SnsTopicArn": topic_arn
      }
    },
    AlertDescription = "Test Alert 1",
    AlertName = project + "-alert-1",
    AnomalyDetectorArn = anomaly_detector_arn,
    AlertSensitivityThreshold = 50
)

alert_arn = response["AlertArn"]
alert_arn

That's it! 

You've officially created a `Continuous` detector with the APIs and should be receiving alerts going forward via the SNS topic that you created. The cell below will delete the resources you created in this notebook. Only run it when you are totally ready to delete everything.

## Clean up resources 

You can start to cleanup the resources that were created. 

Note this will erase all the resources that have been created, so wait to run this until you are sure you wish to delete everything.

Feel free to also continue to explore with the markdown files for how to do these tasks in the console as well.

In [ ]:
answer = input("Delete resources? (y/n)")
if answer=="y":
    delete_resources = True
else:
    delete_resources = False
    
if delete_resources:
    L4M.delete_anomaly_detector( AnomalyDetectorArn = anomaly_detector_arn )
    utility.wait_delete_anomaly_detector( L4M, anomaly_detector_arn )
    utility.delete_iam_role(role_name)
else:
    print("Not deleting resources.")